In [15]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import numpy as np
# import StringIO
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
np.random.seed(7)

In [18]:
%run ../../prepare_data.py

# import sys
# sys.path.insert(0, '../../')
# import preprare_data

In [22]:
additional_data = [el[2] for el in content]

possible_variants = sorted(set(additional_data))

In [141]:
def decode_X_features(feature_dictionary, X_other_features):
    for word in X_other_features:
        final_word = []
        i = 0
        for z in range(len(feature_dictionary)):
            for j in range(1, len(feature_dictionary[z])):
                if j == 1:
                    if word[i] == 1:
#                         print feature_dictionary[z][1]
                        final_word.append(feature_dictionary[z][1])
                    i += 1
                else:
                    for k in range(len(feature_dictionary[z][j])):
                        if word[i] == 1:
#                             print feature_dictionary[z][j][k]
                            final_word.append(feature_dictionary[z][j][k])
                        i += 1
        print(u''.join(final_word))
decode_X_features(feature_dictionary, X_other_features)

Vmps-sna
Vmip1s--n
Vmip1p--n
Vmip2s--n
Vmu
Vmip2d--n
Vmip3d--n
Vmip2p--n
Vmn
Vmip1d--n
Rgp
Afpmsnn


In [147]:
np.array(X_other_features).shape

(536431, 167)

In [142]:
test = 3

def create_X_features(content, feature_dictionary):
    content = content
    X_other_features = []
    for el in content:
        X_el_other_features = []
        for feature in feature_dictionary:
            if el[2][0] == feature[1]:
                X_el_other_features.append(1)
                for i in range(2, len(feature)):
                    for j in range(len(feature[i])):
                        if i-1 < len(el[2]) and feature[i][j] == el[2][i-1]:
                            X_el_other_features.append(1)
                        else:
                            X_el_other_features.append(0)
#                 X_other_features.append(X_el_other_features)
                        
            else:
                X_el_other_features.extend([0] * feature[0])
#         break
#         print X_el_other_features
        X_other_features.append(X_el_other_features)
    return X_other_features
        
X_other_features = create_X_features(content, feature_dictionary)

In [46]:
def create_feature_dictionary(content):
    additional_data = [el[2] for el in content]
    possible_variants = sorted(set(additional_data))
    categories = sorted(set([el[0] for el in possible_variants]))
#     print categories

    # [all_atts[i] for i in range(1,len(all_atts))]

    specifics = [el for el in possible_variants if el[0] == 'A']
#     print sorted(set([el[1] for el in specifics if el]))
    # len(possible_variants)
    # len(specifics)
    # print specifics
    feature_dictionary = []
    for category in categories:
        category_features = [1, category]
        examples_per_category = [el for el in possible_variants if el[0] == category]
        longest_element = max(examples_per_category, key=len)
        for i in range(1, len(longest_element)):
            possibilities_per_el = sorted(set([el[i] for el in examples_per_category if i < len(el)]))
            category_features[0] += len(possibilities_per_el)
            category_features.append(possibilities_per_el)
        feature_dictionary.append(category_features)
    #     print longest_element
    return feature_dictionary

feature_dictionary = create_feature_dictionary(content)
feature_dictionary

[[25,
  u'A',
  [u'f', u'o', u's'],
  [u'c', u'p', u's'],
  [u'f', u'm', u'n'],
  [u'd', u'p', u's'],
  [u'a', u'd', u'g', u'i', u'l', u'n'],
  [u'-', u'n', u'y'],
  [u'-'],
  [u'n', u'y']],
 [5, u'C', [u'c', u's'], [u'c', u's']],
 [1, u'I'],
 [25,
  u'M',
  [u'c', u'm', u'o', u's'],
  [u'f', u'm', u'n'],
  [u'd', u'p', u's'],
  [u'a', u'd', u'g', u'i', u'l', u'n'],
  [u'l'],
  [u'-', u'n', u'y'],
  [u'-'],
  [u'-'],
  [u'n', u'y']],
 [18,
  u'N',
  [u'c'],
  [u'f', u'm', u'n'],
  [u'd', u'p', u's'],
  [u'a', u'd', u'g', u'i', u'l', u'n'],
  [u'-'],
  [u'-'],
  [u'n', u'y']],
 [48,
  u'P',
  [u'd', u'g', u'i', u'p', u'q', u'r', u's', u'x', u'z'],
  [u'-', u'1', u'2', u'3'],
  [u'-', u'f', u'm', u'n'],
  [u'-', u'd', u'p', u's'],
  [u'-', u'a', u'd', u'g', u'i', u'l', u'n'],
  [u'-', u'd', u'p', u's'],
  [u'-', u'f', u'm', u'n'],
  [u'-', u'n', u'y'],
  [u'-', u'p', u's'],
  [u'a', u'n'],
  [u'-'],
  [u'n', u'y']],
 [1, u'Q'],
 [5, u'R', [u'g'], [u'c', u'p', u's']],
 [9, u'S', [u'p'], [

In [ ]:
# X, y = generate_inputs()
# save_inputs('ordered_inputs.h5', X, y)
# X, y = load_inputs('ordered_inputs.h5')

# X, y = generate_matrix_inputs()
# save_inputs('ordered_matrix_inputs.h5', X, y)
# X, y = load_inputs('ordered_matrix_inputs.h5')

X, y = generate_full_matrix_inputs()
X_train, y_train, X_validate, y_validate = generate_full_matrix_inputs()
save_inputs('../internal_representations/inputs/shuffeled_full_matrix_train_inputs.h5', X_train, y_train)
save_inputs('../internal_representations/inputs/shuffeled_full_matrix_validate_inputs.h5', X_validate, y_validate)
# X_train, y_train = load_inputs('../internal_representations/inputs/shuffeled_full_matrix_train_inputs.h5')
# X_validate, y_validate = load_inputs('../internal_representations/inputs/shuffeled_full_matrix_validate_inputs.h5')

# X, y, X_pure = generate_full_vowel_matrix_inputs()

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...


In [10]:
# X = X[:100000]
# y = y[:100000]
# def unison_shuffled_copies(a, b):
#     assert len(a) == len(b)
#     p = np.random.permutation(len(a))
#     return a[p], b[p]
# X, y = unison_shuffled_copies(X, y)
# print X.shape

# train_X = X.astype(np.float32)
# train_y = y.astype(np.int32)

num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 121
nn_hdim = 516

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [6]:
nn_output_dim

121

In [11]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(3, 32, 32), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Conv2D(32, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))


np.random.seed(7)


# create model
# model = Sequential()
# model.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(43, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(86, (3), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(86, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(172, (3), padding='same', activation='relu'))
# model.add(Dropout(0.2))
# model.add(Conv1D(172, (3), padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# # model.add(Conv1D(43, (3), input_shape=(None, 43), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Flatten())
# model.add(Dense(1032, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
# model.add(Dense(nn_output_dim, activation='softmax'))


# create model
model = Sequential()
model.add(Conv1D(43, (3), input_shape=(23, 43), padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(Conv1D(43, (3), padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(43, (3), input_shape=(None, 43), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(Flatten())
# model.add(Dense(1032, activation='relu', kernel_constraint=maxnorm(3)))
# model.add(Dropout(0.2))
model.add(Dense(516, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(nn_output_dim, activation='sigmoid'))

In [12]:
epochs = 5
lrate = 0.1
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [13]:
# model.fit_generator((X, y), 536432/2, epochs=5)
# testX = X[:536432/16]
# print testX.shape
# testY = y[:536432/16]
# print testY.shape
model.fit(X_train, y_train, validation_data=(X_validate, y_validate), epochs=3, batch_size=10)
# model.fit(X, y, epochs=5, validation_split=0.2, batch_size=10)

Train on 428142 samples, validate on 108290 samples
Epoch 1/3
428142/428142 [==============================] - 1020s - loss: 0.0022 - acc: 0.8302 - val_loss: 0.0022 - val_acc: 0.8293
Epoch 2/3
428142/428142 [==============================] - 1554s - loss: 0.0011 - acc: 0.9169 - val_loss: 0.0020 - val_acc: 0.8451
Epoch 3/3
  1010/428142 [..............................] - ETA: 1818s - loss: 0.0010 - acc: 0.9158

KeyboardInterrupt: 

In [14]:
model.save('cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_3epoch_no_overfitting.h5')

In [10]:
model = load_model('../internal_representations/models/cnn_i2_s_c43-3relu_d20_c43-3relu_mp2_f_516relu_d20_121sigmoid_mse_adam_a65_10epoch.h5')

In [6]:
h5f.close()

NameError: name 'h5f' is not defined

In [12]:
# evaluate the model
scores = model.evaluate(X[429145:], y[429145:])
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

107232/107287 [============================>.] - ETA: 0s
acc: 96.52%


In [118]:
# calculate predictions
# test = generate_input_from_word('test', max_word, dictionary)
# print test.shape
# print test.T.shape
predictions = model.predict(np.array([generate_input_from_word('biotski', max_word, dictionary)]))
# round predictions

print decode_position(predictions[0], max_num_vowels)

b_pred = 0
ind = 0
i = 0
for el in predictions[0]:
    if b_pred < el:
        b_pred = el
        ind = i
    i += 1
    
print(ind)
# rounded = [round(x[0]) for x in predictions]
# print(rounded)

[2, 0]
2


In [ ]:
predictions = model.predict(X[429145:])

In [21]:
decode_position(predictions[0], max_num_vowels)

[1, 0]

In [23]:
def test_accuracy(predictions, y):
    dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
    num_of_pred = len(predictions)
    num_of_correct_pred = 0
    for i in range(predictions.shape[0]):
        if decode_position(predictions[i], max_num_vowels) == decode_position(y[i], max_num_vowels):
            num_of_correct_pred += 1

    return (num_of_correct_pred/float(num_of_pred)) * 100


print(test_accuracy(predictions, y[429145:]))

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
107287
96.5186835311


In [17]:
predictions.shape

(107287, 121)

In [12]:
print max_num_vowels

11


In [21]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
